In [14]:
pip install retry


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\user102\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import time
import numpy as np
import csv

data_samples = []

max_page = 3
base_url = 'https://suumo.jp/chintai/kanagawa/sc_yokohamashitsurumi/?page={}'

@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return None

start = time.time()
times = []

for page in range(1, max_page + 1):
    before = time.time()
    soup = load_page(base_url.format(page))
    
    if soup:
        mother = soup.find_all(class_='cassetteitem')
        for child in mother:
            data_home = []
            data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
            data_home.append(child.find(class_='cassetteitem_content-title').text)
            data_home.append(child.find(class_='cassetteitem_detail-col1').text)
            children = child.find(class_='cassetteitem_detail-col3')
            for grandchild in children.find_all('div'):
                data_home.append(grandchild.text)
            rooms = child.find(class_='cassetteitem_other')
            for room in rooms.find_all(class_='js-cassette_link'):
                data_room = []
                for id_, grandchild in enumerate(room.find_all('td')):
                    if id_ == 2:
                        data_room.append(grandchild.text.strip())
                    elif id_ == 3:
                        data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    elif id_ == 5:
                        data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                data_sample = data_home + data_room
                data_samples.append(data_sample)
        time.sleep(1)
        after = time.time()
        running_time = after - before
        times.append(running_time)
        print(f'{page}ページ目：{running_time}秒')
        print(f'総取得件数：{len(data_samples)}')
        complete_ratio = round(page / max_page * 100, 3)
        print(f'完了：{complete_ratio}%')
        running_mean = np.mean(times)
        running_required_time = running_mean * (max_page - page)
        hour = int(running_required_time / 3600)
        minute = int((running_required_time % 3600) / 60)
        second = int(running_required_time % 60)
        print(f'残り時間：{hour}時間{minute}分{second}秒\n')

finish = time.time()
running_all = finish - start
print('総経過時間：', running_all)

csv_file_name = 'suumo.csv'

with open(csv_file_name, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['カテゴリ', '建物名', '住所', '築年数', '階数', '階', '家賃', '間取り'])
    writer.writerows(data_samples)


1ページ目：2.655231475830078秒
総取得件数：107
完了：33.333%
残り時間：0時間0分5秒

2ページ目：2.40224027633667秒
総取得件数：161
完了：66.667%
残り時間：0時間0分2秒

3ページ目：2.6175379753112793秒
総取得件数：262
完了：100.0%
残り時間：0時間0分0秒

総経過時間： 7.675238132476807


In [2]:
import pandas as pd

# CSVファイルを読み込む
df = pd.read_csv('suumo.csv')

# カラムの順序を変更する
# 例: '家賃' カラムを先頭に移動する場合
new_order = ['カテゴリ', '建物名', '住所', '間取り', '家賃', '階']
df = df[new_order]

# 変更されたカラム順序でCSVファイルに再保存（エンコーディングを utf-8-sig に設定）
df.to_csv('suumo_tsurumi-1-1206_reordered.csv', index=False, encoding='utf-8-sig')


In [3]:
##new_order = ['カテゴリ', '建物名', '住所', '築年数', '間取り', '家賃', '階数', '階']